# Implementación de un Sistema Experto de Animales

Un ejemplo del [Currículo de IA para Principiantes](http://github.com/microsoft/ai-for-beginners).

En este ejemplo, implementaremos un sistema basado en conocimiento sencillo para determinar un animal en función de algunas características físicas. El sistema puede representarse mediante el siguiente árbol AND-OR (esto es una parte del árbol completo, podemos añadir más reglas fácilmente):

![](../../../../lessons/2-Symbolic/images/AND-OR-Tree.png)


## Nuestro propio sistema experto con inferencia hacia atrás

Intentemos definir un lenguaje simple para la representación del conocimiento basado en reglas de producción. Usaremos clases de Python como palabras clave para definir reglas. Básicamente, habrá 3 tipos de clases:
* `Ask` representa una pregunta que necesita ser hecha al usuario. Contiene el conjunto de posibles respuestas.
* `If` representa una regla, y es simplemente una forma sintáctica de almacenar el contenido de la regla.
* `AND`/`OR` son clases para representar las ramas AND/OR del árbol. Solo almacenan la lista de argumentos dentro. Para simplificar el código, toda la funcionalidad se define en la clase padre `Content`.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

En nuestro sistema, la memoria de trabajo contendría la lista de **hechos** como **pares atributo-valor**. La base de conocimiento puede definirse como un gran diccionario que asigna acciones (nuevos hechos que deben insertarse en la memoria de trabajo) a condiciones, expresadas como expresiones AND-OR. Además, algunos hechos pueden ser `Preguntados`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Para realizar la inferencia hacia atrás, definiremos la clase `Knowledgebase`. Esta contendrá:
* `memory` de trabajo: un diccionario que asigna atributos a valores.
* `rules` de la base de conocimiento en el formato definido anteriormente.

Los dos métodos principales son:
* `get` para obtener el valor de un atributo, realizando la inferencia si es necesario. Por ejemplo, `get('color')` obtendría el valor de un espacio de color (preguntará si es necesario y almacenará el valor para uso posterior en la memoria de trabajo). Si preguntamos `get('color:blue')`, pedirá un color y luego devolverá un valor de `y`/`n` dependiendo del color.
* `eval` realiza la inferencia propiamente dicha, es decir, recorre el árbol AND/OR, evalúa subobjetivos, etc.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Ahora definamos nuestra base de conocimientos sobre animales y realicemos la consulta. Ten en cuenta que esta llamada te hará preguntas. Puedes responder escribiendo `y`/`n` para preguntas de sí o no, o especificando un número (0..N) para preguntas con respuestas de opción múltiple más largas.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Usando PyKnow para Inferencia Hacia Adelante

En el siguiente ejemplo, intentaremos implementar inferencia hacia adelante utilizando una de las bibliotecas para representación del conocimiento, [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** es una biblioteca para crear sistemas de inferencia hacia adelante en Python, diseñada para ser similar al clásico sistema antiguo [CLIPS](http://www.clipsrules.net/index.html).

También podríamos haber implementado el encadenamiento hacia adelante nosotros mismos sin muchos problemas, pero las implementaciones ingenuas generalmente no son muy eficientes. Para un emparejamiento de reglas más efectivo, se utiliza un algoritmo especial llamado [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

Definiremos nuestro sistema como una clase que hereda de `KnowledgeEngine`. Cada regla se define mediante una función separada con la anotación `@Rule`, que especifica cuándo debe activarse la regla. Dentro de la regla, podemos agregar nuevos hechos utilizando la función `declare`, y al agregar esos hechos, se activarán más reglas mediante el motor de inferencia hacia adelante.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Una vez que hemos definido una base de conocimientos, llenamos nuestra memoria de trabajo con algunos hechos iniciales y luego llamamos al método `run()` para realizar la inferencia. Puedes ver como resultado que nuevos hechos inferidos se añaden a la memoria de trabajo, incluyendo el hecho final sobre el animal (si configuramos todos los hechos iniciales correctamente).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
